In [ ]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from time import mktime
import time
import re
from io import StringIO
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [ ]:
def get_data(ticker=str,begin=str,end=str):
    
    import time
    begin_time = time.mktime(datetime.strptime(begin, "%Y%m%d").timetuple())
    end_time = time.mktime(datetime.strptime(end, "%Y%m%d").timetuple())
    url = 'https://finance.yahoo.com/quote/{0}/history?period1={1}&period2={2}&interval=1d&filter=history&frequency=1d'.format(ticker,int(begin_time),int(end_time))
#     url = 'https://finance.yahoo.com/quote/{0}/history?period1=1551398400&period2=1582934400&interval=1d&filter=history&frequency=1d'.format(ticker)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    table_tag = soup.find('table', class_='W(100%) M(0)')
    table_tag = BeautifulSoup(str(table_tag), 'html.parser')
    rows = table_tag.find_all('tr', attrs={'class':'BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)'})

    df = pd.DataFrame()
    for row in rows:
        time = row.find_all('td', attrs={'class':'Py(10px) Ta(start) Pend(10px)'})
        cells = row.find_all('td', attrs={'class':'Py(10px) Pstart(10px)'})
    # print(cells)
        try: 
            d = {
            'time' : time[0].text,
            'Open' : cells[0].text,
            'High': cells[1].text,
            'Low' : cells[2].text,
            'Close' : cells[3].text,
            'Adj Close**' : cells[4].text,
            'Ticker' : ticker
            }
        except:
            continue
        df = df.append(d, ignore_index=True) 
    df['time'] = pd.to_datetime(df['time']).dt.strftime('%Y%m%d')
    cols=[i for i in df.columns if i not in ["time",'Ticker']]
    for col in cols:
        df[col]=pd.to_numeric(df[col])
    df = df.sort_values('time', ascending = True).reset_index(drop=True)
    return df  
    

In [ ]:
def download_data(tickers,begin=str,end=str):
    df = pd.DataFrame()
    for tick in tickers:
        df2 = get_data(tick,begin,end)
        df = pd.concat([df,df2])
    return df

In [7]:
tickers = ['ALFEN.AS','AF.PA','BESI.AS','SLR.MC']

In [8]:
df = download_data(tickers,'20191001','20200630')

In [11]:
df[df['Ticker']]

KeyError: "None of [Index(['ALFEN.AS', 'ALFEN.AS', 'ALFEN.AS', 'ALFEN.AS', 'ALFEN.AS', 'ALFEN.AS',\n       'ALFEN.AS', 'ALFEN.AS', 'ALFEN.AS', 'ALFEN.AS',\n       ...\n       'SLR.MC', 'SLR.MC', 'SLR.MC', 'SLR.MC', 'SLR.MC', 'SLR.MC', 'SLR.MC',\n       'SLR.MC', 'SLR.MC', 'SLR.MC'],\n      dtype='object', length=399)] are in the [columns]"